In [ ]:
%pip install llama-index

In [ ]:
%pip install llama-index-llms-openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"

import nest_asyncio
nest_asyncio.apply()

In [ ]:
#copied imports from llama-index example for subquestion
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

#copied imports from llama-index example for multistep
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

In [ ]:
# import os
from llama_index.core import SimpleDirectoryReader
import urllib.request

os.makedirs('data/paul_graham/', exist_ok=True)

# Download the file
url = 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt'
file_path = 'data/paul_graham/paul_graham_essay.txt'
urllib.request.urlretrieve(url, file_path)

# load documents
pg_essay = SimpleDirectoryReader("./data/paul_graham/").load_data()


In [ ]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    pg_essay,
    use_async=True,
).as_query_engine()

In [ ]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="pg_essay",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [ ]:
llama_debug.flush_event_logs() #flush for easier retrieval of subquestions

response = query_engine.query(
    "Summarize the essay, and discuss the authors life before and after the essay was written."
) #todo: figure out how to stop it from printing responses

In [ ]:
from llama_index.core.callbacks import CBEventType

event_pairs = llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)

sub_questions = []
for pair in event_pairs:
    sub_q_object = str(pair[0].payload["sub_question"])
    start_index = sub_q_object.find("sub_question=")
    end_index = sub_q_object.find(", tool_name=")
    if start_index == -1 or end_index == -1:
        # smth wrong
        continue

    new_q = sub_q_object[start_index+14:end_index-1]
    sub_questions.append(new_q)

# probably a better way to do this than to substring

    

In [ ]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(llm=gpt35, verbose=True)

index_summary = "Used to answer questions about the author"

In [ ]:
from llama_index.core.query_engine import MultiStepQueryEngine

mqe = VectorStoreIndex.from_documents(documents).as_query_engine(llm=gpt4)

multi_query_engine = MultiStepQueryEngine(
    query_engine=mqe,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)

for sub_q in sub_questions:
    print("query: ", sub_q)
    response = multi_query_engine.query(sub_q)
    # print(response)